In [1]:
import lightfm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

/Users/gui/.virtualenvs/tfrecs/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
from lightfm.datasets import fetch_stackexchange

data = fetch_stackexchange('crossvalidated',
                           test_set_fraction=0.1,
                           indicator_features=False,
                           tag_features=True)

data['test_positive_only'] = data['test'].copy()
train = data['train']
test = data['test']
test_positives = data['test_positive_only']

print(np.unique(data['train'].data))
print(data['train'].__repr__())
print(np.unique(data['test'].data))
print(data['test'].__repr__())
print(np.unique(data['test_positive_only'].data))
print(data['test_positive_only'].__repr__())

item_features = data['item_features']
tag_labels = data['item_feature_labels']
print('There are %s distinct item features, with values like %s.' % (item_features.shape[1], tag_labels[:3].tolist()))

[ 1.  2.  3.  4.  5.  6.  8.]
<3221x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 57830 stored elements in COOrdinate format>
[ 1.  2.]
<3221x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 4307 stored elements in COOrdinate format>
[ 1.  2.]
<3221x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 4307 stored elements in COOrdinate format>
There are 1246 distinct item features, with values like ['bayesian', 'prior', 'elicitation'].


In [28]:
from lightfm.datasets import fetch_movielens

data = fetch_movielens('movielens', indicator_features=False, genre_features=True)

print('original train')
print(np.unique(data['train'].data))
print(data['train'].__repr__())
print('original test')
print(np.unique(data['test'].data))
print(data['test'].__repr__())

# binarizing traing examples as in the original lightfm paper to use the logistic loss
data['train'].data = np.array([-1, 1])[1 * (data['train'].data >= 4)]
data['test'].data = np.array([-1, 1])[1 * (data['test'].data >= 4)]

# should keep only positive test interactions
data['test_positive_only'] = data['test'].copy()
data['test_positive_only'].data = 1 *(data['test_positive_only'].data>=1)
data['test_positive_only'].eliminate_zeros()

train = data['train']
test = data['test']
test_positives = data['test_positive_only']

print('train')
print(np.unique(data['train'].data))
print(data['train'].__repr__())
print('test')
print(np.unique(data['test'].data))
print(data['test'].__repr__())
print('test_positive_only')
print(np.unique(data['test_positive_only'].data))
print(data['test_positive_only'].__repr__())

item_features = data['item_features']
tag_labels = data['item_feature_labels']
print('There are %s distinct item features, with values like %s.' % (item_features.shape[1], tag_labels[:3].tolist()))

original train
[1 2 3 4 5]
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
original test
[1 2 3 4 5]
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>
train
[-1  1]
<943x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 90570 stored elements in COOrdinate format>
test
[-1  1]
<943x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 9430 stored elements in COOrdinate format>
test_positive_only
[1]
<943x1682 sparse matrix of type '<class 'numpy.int64'>'
	with 5469 stored elements in COOrdinate format>
There are 19 distinct item features, with values like ['genre:unknown', 'genre:Action', 'genre:Adventure'].


In [29]:
from lightfm import LightFM

NUM_EPOCHS = 3
cf_model = LightFM(
    loss='warp',
    item_alpha=1e-6,
    no_components=10)

%time model = cf_model.fit(train, epochs=NUM_EPOCHS)

CPU times: user 132 ms, sys: 1.82 ms, total: 134 ms
Wall time: 133 ms


In [30]:
def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))
    
def log_loss(df):
    return -((df.rating == 1) * np.log(df.predicted_rating) + (df.rating == -1) * np.log(1 - df.predicted_rating))

def predicted_df(mat, predicted_values):
    return pd.DataFrame.from_dict({
        'user': mat.row,
        'item': mat.col,
        'rating': mat.data,
        'predicted_rating': predicted_values})\
    .sort_values('user')\
    .assign(log_loss=log_loss)

def model_log_losses_df(model, test_mat, item_features=data['item_features']):
    return predicted_df(
        test_mat, 
        sigmoid(cf_model.predict(test_mat.row, test_mat.col, item_features=item_features)))

predicted_train_df = predicted_df(train, sigmoid(cf_model.predict(train.row, train.col)))
print(predicted_train_df.log_loss.mean())
print(predicted_train_df.shape)

predicted_test_df = predicted_df(test, sigmoid(cf_model.predict(test.row, test.col)))
print(predicted_test_df.log_loss.mean())
print(predicted_test_df.shape)
print(model_log_losses_df(cf_model, test_positives).log_loss.mean())
predicted_test_df.head()

0.753395016575
(90570, 5)
0.733679104411
(9430, 5)
0.742460669635


,item,predicted_rating,rating,user,log_loss
0,19,0.154877,1,0,1.865122
1,32,0.209946,1,0,1.560903
2,60,0.218307,1,0,1.521852
3,116,0.695923,-1,0,1.190473
4,154,0.164430,-1,0,0.179642


In [35]:
from lightfm.evaluation import reciprocal_rank, auc_score

cf_model = LightFM(loss='logistic', item_alpha=1e-6, no_components=30)
cf_model.fit(train, epochs=NUM_EPOCHS)

print('Collaborative filtering train/test MRR: %.3f / %.3f'
      % (reciprocal_rank(cf_model, train).mean(),
         reciprocal_rank(cf_model, test).mean()))

print('Collaborative filtering train/test AUC: %.3f / %.3f'
      % (auc_score(cf_model, train).mean(),
         auc_score(cf_model, test, train_interactions=None).mean()))

Collaborative filtering train/test MRR: 0.602 / 0.167
Collaborative filtering train/test AUC: 0.649 / 0.639


In [8]:
def sparse_to_df(mat):
    coo_mat = mat.tocoo()
    return pd.DataFrame({
        'user': coo_mat.row,
        'item': coo_mat.col,
        'rank': coo_mat.data}        
    )

def model_to_mrr(model, test_mat):
    predict_kwargs = {}
    if data['item_features'].shape[1] == model.item_embeddings.shape[0]:
        predict_kwargs['item_features'] = data['item_features']
    #if data['user_features'].shape[1] == model.user_embeddings.shape[0]:
    #    predict_kwargs['user_features'] = data['user_features']
        
    return sparse_to_df(model.predict_rank(test_mat, **predict_kwargs))\
        .assign(rec_rank=lambda df:1 / (df['rank'] + 1))\
        .groupby('user')['rec_rank'].max()\
        .mean()

test_ranks = cf_model.predict_rank(test)
test_predicted_ranks_df = sparse_to_df(test_ranks)
print(test_predicted_ranks_df.shape)
print(model_to_mrr(cf_model, test))
test_predicted_ranks_df.head()

(9430, 3)
0.266324


,item,rank,user
0,19,381.0,0
1,32,324.0,0
2,60,353.0,0
3,116,70.0,0
4,154,439.0,0


In [9]:
(np.sum(test_ranks.sum(axis=1).A > 0), np.sum(test_ranks.sum(axis=0).A > 0)), test_ranks.shape, test_ranks.nnz

((943, 1129), (943, 1682), 9430)

In [19]:
test_ranks

<943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 9430 stored elements in Compressed Sparse Row format>

## With item features

In [119]:
model = LightFM(loss='warp',
                item_alpha=1e-6,
                no_components=30)

model = model.fit(
    data['train'],
    item_features=data['item_features'],
    epochs=3)

In [120]:
print(model_log_losses_df(model, train).log_loss.mean())
print(model_log_losses_df(model, test).log_loss.mean())
print(model_log_losses_df(model, test_positives).log_loss.mean())

0.190722790341
0.197180703211
0.197180703211


In [121]:
from lightfm.evaluation import reciprocal_rank, auc_score

print('Collaborative filtering train/test MRR: %.3f / %.3f'
      % (reciprocal_rank(model, train, item_features=data['item_features']).mean(),
         reciprocal_rank(model, test, item_features=data['item_features']).mean()))

print('Collaborative filtering train/test AUC: %.3f / %.3f'
      % (auc_score(model, train, item_features=data['item_features']).mean(),
         auc_score(model, test, item_features=data['item_features'], train_interactions=train).mean()))

Collaborative filtering train/test MRR: 0.026 / 0.012
Collaborative filtering train/test AUC: 0.855 / 0.712
